In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from mobilenet_v3_small import MobileNetV3_Small
from keras.optimizers import Adam
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
shape = (None, None, 3)
model1 = MobileNetV3_Small(shape, 100).build()
model1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 1 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 1 0           batch_normalization_1[0][0]      
___________

In [3]:
from keras.layers import Input, Lambda
from keras.models import Model
from keras.utils import Sequence
from keras import backend as K

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def get_siamese_model(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
#     model = Sequential()
    
    encoded_l = model1(left_input)
    encoded_r = model1(right_input)
    L1_layer = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([encoded_l, encoded_r])
#     prediction = Dense(1,activation='sigmoid')(L1_layer)
    siamese_net = Model(inputs=[left_input,right_input],outputs=L1_layer)
    return siamese_net
model = get_siamese_model(shape)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 100)          3167732     input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]              

In [4]:
from os import listdir
from os.path import isfile, join
import pandas as pd
dir = '../images/train/'

onlyfiles = [f for f in listdir('../images/train') if isfile(join('../images/train', f))]
dataset = pd.DataFrame()
for i in range(int(min(onlyfiles).split('_')[0]), int(max(onlyfiles).split('_')[0])+1):
    dataset[i] = [f for f in filter(lambda x: int(x.split('_')[0]) == i, onlyfiles)]

In [5]:
def couples(size):
    x = np.empty((0, 2), str)
    y = np.empty((0, 1), int)
    tmp0_0 = np.random.randint(0, len(dataset), size)
    tmp0_1 = np.random.randint(0, len(dataset), size)
    tmp1_0 = np.random.randint(0, len(dataset.columns), size)
    tmp1_1 = np.random.randint(0, len(dataset.columns), size)
    for i in range(0, size):
        da = np.array([dataset[tmp1_0[i]][tmp0_0[i]],
                        dataset[tmp1_1[i]][tmp0_1[i]]])
        x = np.append(x, da)
        y = np.append(y, 1 if tmp0_0[i] == tmp1_0[i] else 0)
    x = x.reshape(size, 2)
    df = pd.DataFrame(x)
    df[2] = pd.Series(y)
    return df
train = couples(1024)
val = couples(102)
# x_train, y_train = couples(1024)
# x_val, y_val = couples(2)
print(train)

                     0                 1  2
0     014_08_image.png  011_06_image.png  0
1     003_04_image.png  020_07_image.png  0
2     018_03_image.png  025_02_image.png  0
3     013_09_image.png  011_09_image.png  0
4     003_09_image.png  014_03_image.png  0
...                ...               ... ..
1019  019_09_image.png  015_12_image.png  0
1020  017_11_image.png  012_06_image.png  0
1021  021_01_image.png  021_00_image.png  0
1022  007_09_image.png  014_00_image.png  0
1023  021_06_image.png  013_06_image.png  0

[1024 rows x 3 columns]


In [7]:
generator = ImageDataGenerator(rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True) 


In [8]:
class DoubleGenerator(Sequence):
    def __init__(self, gen1, gen2):
        self.gen1 = gen1
        self.gen2 = gen2

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        x1,y = self.gen1[i]
        x2,y2 = self.gen2[i]
        print(np.array([x1,x2]).shape, y)
        return [x1,x2], y

double_gen = DoubleGenerator(generator.flow_from_dataframe(train, dir, x_col=0, y_col=2, class_mode='raw', batch_size=256),
                             generator.flow_from_dataframe(train, dir, x_col=0, y_col=2, class_mode='raw', batch_size=256))

Found 1024 validated image filenames.
Found 1024 validated image filenames.


In [9]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
model.fit_generator(double_gen, steps_per_epoch=16, epochs=1, verbose = 1)

Epoch 1/1
(2, 256, 256, 256, 3) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


KeyboardInterrupt: 

In [ ]:
import os.path
import pandas as pd

test_df = pd.DataFrame(columns=['path', 'label'])

with open('images/wider_face_train_bbx_gt.txt') as file:
    while (file):
        fname = file.readline()[:-1]
        try:
            length = int(file.readline())
            if length == 0:
file.readline()
continue
except:
df.to_csv('images/train.csv', index = False, header=True)
break
for i in range(0, length):
test_df = test_df.append({'path': fname, 'label': [int(k) for k in file.readline().split(' ')[0:4]]}, ignore_index=True)
print(test_df)    